In [41]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import operator

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))

C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [4]:
window_size=3
stemmer = nltk.wordnet.WordNetLemmatizer()

In [136]:
def rank(text):
    text=text.lower()
    stops = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    filtered_words = [stemmer.lemmatize(word) for word in tokens if word not in stops]
    scores=dict()
    counts=dict()
    contextSizes=dict()
    for i in range(len(filtered_words)):
        counts.setdefault(filtered_words[i],0)
        counts[filtered_words[i]]+=1
        leftBound=max(0,i-window_size)
        rightBound=min(len(filtered_words)-1,i+window_size)
        contextSize=rightBound-leftBound+1
        contextSizes.setdefault(filtered_words[i],contextSize)
        contextSizes[filtered_words[i]]=max(contextSizes[filtered_words[i]],contextSize)
        for j in range(i+1,rightBound+1):
            scores.setdefault(filtered_words[i],0)
            scores.setdefault(filtered_words[j],0)
            similarity_score=model.wv.similarity(filtered_words[i],filtered_words[j])
            scores[filtered_words[i]]+=similarity_score
            scores[filtered_words[j]]+=similarity_score
    wordScores=list(map(list, scores.items()))
    for i in range(len(wordScores)):
        wordScores[i][1]=counts[wordScores[i][0]]*wordScores[i][1]/(contextSizes[wordScores[i][0]])
    wordScores=sorted(wordScores, key = lambda x: x[1],reverse=True)
    print(contextSizes)
    return wordScores

In [138]:
rank("Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to perform a specific task without using explicit instructions, relying on patterns and inference instead. It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop a conventional algorithm for effectively performing the task.")

{'machine': 7, 'learning': 7, '(': 6, 'ml': 7, ')': 7, 'scientific': 7, 'study': 7, 'algorithm': 7, 'statistical': 7, 'model': 7, 'computer': 7, 'system': 7, 'use': 7, 'perform': 7, 'specific': 7, 'task': 7, 'without': 7, 'using': 7, 'explicit': 7, 'instruction': 7, ',': 7, 'relying': 7, 'pattern': 7, 'inference': 7, 'instead': 7, '.': 7, 'seen': 7, 'subset': 7, 'artificial': 7, 'intelligence': 7, 'build': 7, 'mathematical': 7, 'based': 7, 'sample': 7, 'data': 7, 'known': 7, 'training': 7, 'order': 7, 'make': 7, 'prediction': 7, 'decision': 7, 'explicitly': 7, 'programmed': 7, 'used': 7, 'wide': 7, 'variety': 7, 'application': 7, 'email': 7, 'filtering': 7, 'vision': 7, 'difficult': 7, 'infeasible': 7, 'develop': 7, 'conventional': 7, 'effectively': 7, 'performing': 6}


[['algorithm', 2.7868675461837222],
 [',', 1.7290706812803234],
 ['task', 1.5086108488695962],
 ['machine', 1.2603392473288946],
 ['learning', 1.158843059026237],
 ['model', 1.0156403524535043],
 ['perform', 0.7819848123139569],
 ['without', 0.7693937058959689],
 ['.', 0.6753005398703473],
 ['computer', 0.634229587657111],
 ['data', 0.5291105210781097],
 ['statistical', 0.27330892852374483],
 ['mathematical', 0.22657029437167303],
 ['specific', 0.22493657044001988],
 ['make', 0.22069273782627924],
 ['system', 0.21644370683601924],
 ['explicit', 0.21546707728079387],
 ['use', 0.2118303371327264],
 ['infeasible', 0.2078171255333083],
 ['based', 0.20232183699096953],
 ['decision', 0.19743264679397857],
 ['using', 0.19595717319420405],
 ['study', 0.19502664250986917],
 ['effectively', 0.1934394815138408],
 ['relying', 0.1860872661428792],
 ['build', 0.18284886011055537],
 ['(', 0.18081439721087614],
 [')', 0.1798313202868615],
 ['explicitly', 0.17673254225935256],
 ['instead', 0.1735897617